<a href="https://colab.research.google.com/github/suhailsardar/TextToImageGenerator/blob/main/TextToImageGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyngrok


In [2]:
from pyngrok import ngrok
!ngrok authtoken 2kqkccovOpLm6nYBEUcj43fbjqJ_4beHcBFT3D5JGyVTUQJ9G  # Replace token with the actual token


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
!pip install diffusers torch torchvision streamlit  numpy huggingface-hub
# !pip install -r /content/requirements_clean.txt  (In case if you want to install all libraries from requirement.txt instead of above code in notebook)
# !pip install pillow==10.4.0


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [4]:
%%writefile app_1.py
import streamlit as st
from diffusers import DiffusionPipeline
import torch
from PIL import Image
import io
import numpy as np
import time

@st.cache_resource
def load_model():
    model = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5")
    model.to("cuda" if torch.cuda.is_available() else "cpu")
    return model

generator = load_model()

st.title('Text to Image Generator')
prompt = st.text_input("Enter your prompt:", "A scenic landscape painting")

if st.button('Generate Image'):
    start_time = time.time()  # Start timing
    with st.spinner('Generating image...'):
        # Generate the image with a low number of inference steps
        output = generator(prompt, num_inference_steps=5, eta=0.0)  # Adjust ETA to possibly reduce computation
        image = output.images[0]

        # Convert to NumPy array
        image = np.array(image)

        # Convert to PIL Image
        pil_image = Image.fromarray(image.astype(np.uint8))  # Cast to uint8 for PIL compatibility

        # Display the image
        st.image(pil_image, caption='Generated Image', use_column_width=True)

        # Provide a download button
        buf = io.BytesIO()
        pil_image.save(buf, format="JPEG")
        byte_im = buf.getvalue()
        st.download_button(
            label="Download Image",
            data=byte_im,
            file_name="generated_image.jpg",
            mime="image/jpeg"
        )

    # Record and display time taken
    end_time = time.time()
    st.write(f"Time taken to generate image: {end_time - start_time:.2f} seconds")

st.write("Enter a textual prompt and press generate to create an image.")


Writing app_1.py


In [5]:

!streamlit run app_1.py &>/dev/null&


In [6]:
url = ngrok.connect(8501)
print("Access your Streamlit app at:", url)

Access your Streamlit app at: NgrokTunnel: "https://1fca-34-86-128-88.ngrok-free.app" -> "http://localhost:8501"


In [7]:
active_tunnels = ngrok.get_tunnels()
print(active_tunnels)


[<NgrokTunnel: "https://1fca-34-86-128-88.ngrok-free.app" -> "http://localhost:8501">]
